# Введение

В ходе выполнения первых 4 домашних заданий "Формальные языки" было реализовано 2 алгоритма для выполнения регулярных запросов к графам.
В основе первого алгоритма лежит идея нахождения пересечения графа и регулярного выражения как конечных автоматов, с послеюдующим нахождением транзитивного замыкания для полученного графа.
В основе второго алгоритма лежит идея синхронного обхода в ширину графа и конечного автомата, задающего регулярное выражение.

Целью данной работы является сравнение полученных решений с целью определить, какая из 2 реализаций регулярных запросов к графам является более быстрой при различных начальных условиях.

# Описание исследуемых решений

### Алгоритм 1

##### Идея

Так как мы имеем граф с метками на ребрах, множество начальных вершин и множество конечных вершин, мы можем рассматривать граф как конечный автомат с начальными и конечными состояниями, задающий некоторый язык. На этом наблюдении основан первый алгоритм:
1. Представляем граф как конечный автомат
2. Представляем регулярное выражение, задающее регулярный запрос, как конечный автомат
3. Находим пересечение этих автоматов
4. Находим все пары состояний в полученном на 3 шагу автомате, между которыми можно построить путь

В автомате, который получается на 3 шаге алгоритма, состояниями являются пары вида $Q_I = \{(u, v) | u \in Q_G, v \in Q_R \}$, где $Q_G$ - набор состояний графа как конечного автомата, а $Q_R$ - набор состояний регулярного выражения как конечного автомата. По построению, если между двумя состояниями $q_1 = (u_1, v_1), q_2 = (u_2, v_2) \in Q_I$ существует путь, то существует путь и между состояниями $u_1, u_2$ в графе, а также между состояниями $v_1, v_2$ в регулярном выражении. Таким образом, когда мы на 4 шагу находим транзитивное замыкание пересечения языков графа и регулярного выражения как конечного автомата, мы получаем всю необходимую информацию для ответа на регулярный запрос.

###### Детали реализации

Автоматы, представляющие граф и регулярное выражение, хранятся в памяти как булева декомпозиция. Автомат, представляющий язык пересечение графа и регулярного выражения, получается как произведение Кронекера для соответсвующих матриц внутри булевых декомпозиций графа и регулярного выражения. Для нахождения пар всех достижимых состояний внутри автомата-пересечения используется матричное умножение матрицы смежности автомата-пересечения на саму себя (таким образом находится транзитивное замыкание).

### Алгоритм 2

##### Идея

По шагам, второй алгоритм можно расписать так:
1. Фиксируем пары $(u, v)$, где $u$ - начальная вершина графа, а $v$ - некоторое начальное состояние регулярного выражения как конечного автомата, и все такие пары добавляем в множество $Query$
2. Из множества пар $Query$, запускаем поиск в ширину, то есть находим все такие пары $(u_i, v_j)$, что $u_i \in Adj(u)$, $v_j \in Adj(v)$, где $Adj(u)$ - множество состояний, смежных с $u$. Все найденные пары мы добавляем в множество $Query$, а все отработанные вершины добавляем в множество $Visited$
3. Пока разность множеств $Query$ и $Visited$ не пуста, повторяем 2 шаг
4. В конце мы получим множество $Visited$ в котором будут все достижимые из начального множества вершины

##### Детали реализации

Также, как и в 1 алгоритме, автоматы, представляющие граф и регулярное выражение, хранятся в памяти как булева декомпозиция. Непосредственно сам обход в ширину производится как умножение матрицы "фронта обхода" (которая является матричным выражением множества $Query$), на каждую из матриц прямой суммы булевых декомпозиций графа и регулярного выражения. Множество $Visited$ тоже выражается как блочная матрица $Visited = [I_n M_{n*k}]$, где $I_n$ - единичная матрица n * n (n - количество состояний в автомате, представляющем регулярное выражение), а $M_{n*k}$ - такая матрица, что если $M[i, j] != 0$, то в состояние $i$ автомата-регулярного выражения можно прийти из начального множества по такому пути, что он закончится в вершине $j$ графа.


# Постановка задачи

Данная работа призвана ответить на следующие вопросы:
1. Какое представление разреженных матриц лучше подходит для каждой реализации алгоритма?
2. Начиная с какого размера начального множества задачу нахождения всех пар достижимых вершин выгоднее решать используя 1 алгоритм, чем 2 алгоритм?
3. На сколько реализация 2 алгоритм медленнее работает, если необходимо найти множества достижимых вершин для каждой из вершин начального множества?

# Набор данных для эксперимента

Для проведения экспериментов были использованы графы из [датасета](https://jetbrains-research.github.io/CFPQ_Data/dataset/index.html), а именно:
* [skos](https://jetbrains-research.github.io/CFPQ_Data/dataset/data/skos.html#skos)
* [bzip](https://jetbrains-research.github.io/CFPQ_Data/dataset/data/bzip.html#bzip)
* [funding](https://jetbrains-research.github.io/CFPQ_Data/dataset/data/funding.html#funding)
* [gzip](https://jetbrains-research.github.io/CFPQ_Data/dataset/data/gzip.html#gzip)

Выбор именно этих графов обусловлен тем, что:
1. Хотелось выбрать графы с различными (по размеру) алфавитами меток на ребрах
2. На больших графах, чем выбранные, замеры занимали слишком много времени на имеющемся железе

В качестве регулярных выражений для экспериментов были выбраны 3 вида:
1. Регулярное выражение, допускающее весь графа
2. Регулярное выражение, допускающее один символ из алфавита меток
3. Регулярное выражение, содержащее все основные конструкции регулярных выражений

# Описание эксперимента

### Оборудование

Все тесты производились на следующей конфигурации:
* Процессор
<pre>
Архитектура:             x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         39 bits physical, 48 bits virtual
  Порядок байт:          Little Endian
CPU(s):                  4
  On-line CPU(s) list:   0-3
ID прроизводителя:       GenuineIntel
  Имя модели:            Intel(R) Core(TM) i7-5500U CPU @ 2.40GHz
    Семейство ЦПУ:       6
    Модель:              61
    Потоков на ядро:     2
    Ядер на сокет:       2
    Сокетов:             1
    Степпинг:            4
    CPU max MHz:         3000,0000
    CPU min MHz:         500,0000
    BogoMIPS:            4789.01
    Флаги:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss
                         ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arc
                         h_perfmon pebs bts rep_good nopl xtopology nonstop_tsc
                         cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl vm
                         x est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid sse4_1 sse
                         4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave av
                         x f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb
                          invpcid_single pti ssbd ibrs ibpb stibp tpr_shadow vnm
                         i flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1
                          avx2 smep bmi2 erms invpcid rdseed adx smap intel_pt x
                         saveopt dtherm ida arat pln pts md_clear flush_l1d
Virtualization features:
  Виртуализация:         VT-x
Caches (sum of all):
  L1d:                   64 KiB (2 instances)
  L1i:                   64 KiB (2 instances)
  L2:                    512 KiB (2 instances)
  L3:                    4 MiB (1 instance)
NUMA:
  NUMA node(s):          1
  NUMA node0 CPU(s):     0-3
Vulnerabilities:
  Itlb multihit:         KVM: Mitigation: VMX disabled
  L1tf:                  Mitigation; PTE Inversion; VMX conditional cache flushe
                         s, SMT vulnerable
  Mds:                   Mitigation; Clear CPU buffers; SMT vulnerable
  Meltdown:              Mitigation; PTI
  Mmio stale data:       Not affected
  Retbleed:              Not affected
  Spec store bypass:     Mitigation; Speculative Store Bypass disabled via prctl
                          and seccomp
  Spectre v1:            Mitigation; usercopy/swapgs barriers and __user pointer
                          sanitization
  Spectre v2:            Mitigation; Retpolines, IBPB conditional, IBRS_FW, STIB
                         P conditional, RSB filling, PBRSB-eIBRS Not affected
  Srbds:                 Mitigation; Microcode
  Tsx async abort:       Not affected
</pre>
* Оперативная память - 16 гигабайт

### Методика проведения замеров

Для проведения замеров времени выполнения регулярных запросов была написана функция time, которая позволяет заданное количество раз (в тестах автора это 100 раз) выполнить некоторый кусок кода, а также посчитать среднее время выполнения данного заданного куска кода, а также узнать среднее квадратическое отклонение этого времени.

In [9]:
import random
import statistics

import prettytable
from prettytable import PrettyTable

from project.left_right_matrix import LeftRightMatrix
from timeit import timeit

from networkx import MultiDiGraph
from pyformlang.regular_expression import Regex

from project.graph_utils import from_graph_to_graph_data
from project.regular_path_query import regular_path_query, bfs_based_regular_path_query
from project.boolean_decomposition import BooleanDecomposition

import cfpq_data

def generate_regexes(graph: MultiDiGraph) -> list[Regex]:
    """
    Generates test regexes for given graph
    :param graph: graph, which alphabet must be used for regex
    :return: list or generated regexes
    """
    graph_data = from_graph_to_graph_data(graph)

    def regex1():
        string_regex = "|".join(graph_data.labels)
        return Regex(f'({string_regex})*')

    def regex2():
        string_regex = random.choice(list(graph_data.labels))
        return Regex(f'{string_regex}')

    def regex3():
        list_of_labels = list(graph_data.labels)
        a, b = random.choices(list_of_labels, k=2)
        while b == a:
            b = random.choice(list_of_labels)
        string_regex = f'{a}.{a}*.{b}'
        return Regex(string_regex)

    return [regex1(), regex2(), regex3()]

test_graph_names = ["skos", "bzip", "funding", "gzip"]
test_graphs = list(map(lambda name: cfpq_data.graph_from_csv(cfpq_data.dataset.download(name)), test_graph_names))
test_regexes = list(map(lambda graph: generate_regexes(graph), test_graphs))

execution_times_number = 100
def time(stmt) -> tuple[float, float]:
    """
    Measure time of execution of given statement
    :param stmt: statement which execution time must be measured
    :return: 2 element tuple with mean and standard deviation of measured times
    """
    times = []
    for i in range(execution_times_number):
        times.append(timeit(stmt=stmt, number=1))
    return statistics.mean(times), statistics.stdev(times)


def experiment_pass_regular_path_query(regex: Regex, graph: MultiDiGraph, start_states: list[any] = None, final_states: list[any] = None):
    """
    Statement, which will be measured with time function
    """
    regular_path_query(
        regex,
        graph,
        start_states,
        final_states
    )

def experiment_pass_bfs_based_regular_path_query(regex: Regex, graph: MultiDiGraph, separated: bool, start_states: list[any], final_states: list[any] = None):
    """
    Statement, which will be measured with time function
    """
    bfs_based_regular_path_query(
        regex,
        graph,
        separated,
        start_states,
        final_states
    )

def print_table(header: list[str], rows: list[list[any]]):
    """
    Utility function to print python results as table
    """
    table = PrettyTable()
    table.set_style(prettytable.MARKDOWN)
    table.field_names = header
    for row in rows:
        table.add_row(row)
    table.max_width = 30
    print(table)

[2022-10-14 14:58:33]>INFO>Found graph with name='skos'
[2022-10-14 14:58:33]>INFO>Load archive graph_archive=PosixPath('/home/user/PycharmProjects/formal-lang-course/venv/lib/python3.10/site-packages/cfpq_data/data/skos.tar.gz')
[2022-10-14 14:58:33]>INFO>Unzip graph name='skos' to file graph=PosixPath('/home/user/PycharmProjects/formal-lang-course/venv/lib/python3.10/site-packages/cfpq_data/data/skos/skos.csv')
[2022-10-14 14:58:33]>INFO>Remove archive graph_archive=PosixPath('/home/user/PycharmProjects/formal-lang-course/venv/lib/python3.10/site-packages/cfpq_data/data/skos.tar.gz')
[2022-10-14 14:58:33]>INFO>Load graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> from path=PosixPath('/home/user/PycharmProjects/formal-lang-course/venv/lib/python3.10/site-packages/cfpq_data/data/skos/skos.csv')
[2022-10-14 14:58:33]>INFO>Found graph with name='bzip'
[2022-10-14 14:58:34]>INFO>Load archive graph_archive=PosixPath('/home/user/PycharmProjects/formal-lang-course/

# Эксперимент

### Какое представление разреженных матриц лучше подходит для каждой реализации алгоритма?

В обоих алгоритмах приходится умножать матрицы, и в обоих алгоритмах это основная часть. В связи с этим было решено провести эксперимент с выбором различных представлений матрицы на моменте умножений. Внутри scipy есть два вида матриц для высокопроизводительных операций:
1. [csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)
2. [csc_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_array.html#scipy.sparse.csc_array)

Так как вид регулярного выражения влияет на длительность выполнения функции и количество умножений матриц внутри каждой из реализаций, выберем такое, которое дает наибольшее количество матричных умножений (регулярное выражение, допускающее любое слово, составленное из меток графа). Составим запросы к графам и прведем замеры:

In [10]:
def set_mul_matrix_type_converter(converter):
    """
    :param converter: lambda which will be used before matrix multiplication to convert spmatrix to particular spmatrix format
    """
    BooleanDecomposition._convert_to_spmatrix = converter
    LeftRightMatrix._convert_to_spmatrix = converter

start_states = list(map(lambda graph: cfpq_data.generate_multiple_source(graph, 1), test_graphs)) # in this test, we do not care about the size and composition of the initial set

set_mul_matrix_type_converter(lambda mat: mat.tocsr())
regular_path_query_csr_results = []
bfs_based_regular_path_query_csr_results = []
for i in range(len(test_graphs)):
    graph = test_graphs[i]
    regex = test_regexes[i][0]
    regular_path_query_csr_results.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_regular_path_query(regex, graph, start_states=start_states[i]))))
    bfs_based_regular_path_query_csr_results.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_bfs_based_regular_path_query(regex, graph, separated=False, start_states=start_states[i]))))

set_mul_matrix_type_converter(lambda mat: mat.tocsc())
regular_path_query_csc_results = []
bfs_based_regular_path_query_csc_results = []
for i in range(len(test_graphs)):
    graph = test_graphs[i]
    regex = test_regexes[i][0]
    regular_path_query_csc_results.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_regular_path_query(regex, graph, start_states=start_states[i]))))
    bfs_based_regular_path_query_csc_results.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_bfs_based_regular_path_query(regex, graph, separated=False, start_states=start_states[i]))))

[2022-10-14 14:58:34]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 14:58:34]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8560763310> for multiple-source evaluation
[2022-10-14 14:58:34]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8526d42f80> for multiple-source evaluation
[2022-10-14 14:58:34]>INFO>Generate set of source vertices of 1 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f856073f610> for multiple-source evaluation


И распечатаем результат:

In [11]:
rows = []
for i in range(len(regular_path_query_csc_results)):
    data = regular_path_query_csc_results[i]
    rows.append(["csc", "1", data[0], '%.3f' %data[2][0], '%.3f' %data[2][1]])
    data = regular_path_query_csr_results[i]
    rows.append(["csr", "1", data[0], '%.3f' %data[2][0], '%.3f' %data[2][1]])

for i in range(len(regular_path_query_csc_results)):
    data = bfs_based_regular_path_query_csc_results[i]
    rows.append(["csc", "2", data[0], '%.3f' % data[2][0], '%.3f' %data[2][1]])
    data = bfs_based_regular_path_query_csr_results[i]
    rows.append(["csr", "2", data[0], '%.3f' %data[2][0], '%.3f' %data[2][1]])

print_table(["Matrix type", "Algorithm number", "Graph", "Mean of time", "Standard deviation of time"], rows)

| Matrix type | Algorithm number |  Graph  | Mean of time | Standard deviation of time |
|:-----------:|:----------------:|:-------:|:------------:|:--------------------------:|
|     csc     |        1         |   skos  |    0.146     |           0.003            |
|     csr     |        1         |   skos  |    0.212     |           0.083            |
|     csc     |        1         |   bzip  |    0.214     |           0.013            |
|     csr     |        1         |   bzip  |    0.275     |           0.083            |
|     csc     |        1         | funding |    0.552     |           0.012            |
|     csr     |        1         | funding |    0.557     |           0.015            |
|     csc     |        1         |   gzip  |    3.224     |           0.055            |
|     csr     |        1         |   gzip  |    3.361     |           0.480            |
|     csc     |        2         |   skos  |    0.227     |           0.008            |
|     csr     |      

##### Результат полученный автором
| Matrix type | Algorithm number |  Graph  | Mean of time | Standard deviation of time |
|:-----------:|:----------------:|:-------:|:------------:|:--------------------------:|
|     csc     |        1         |   skos  |    0.146     |           0.003            |
|     csr     |        1         |   skos  |    0.212     |           0.083            |
|     csc     |        1         |   bzip  |    0.214     |           0.013            |
|     csr     |        1         |   bzip  |    0.275     |           0.083            |
|     csc     |        1         | funding |    0.552     |           0.012            |
|     csr     |        1         | funding |    0.557     |           0.015            |
|     csc     |        1         |   gzip  |    3.224     |           0.055            |
|     csr     |        1         |   gzip  |    3.361     |           0.480            |
|     csc     |        2         |   skos  |    0.227     |           0.008            |
|     csr     |        2         |   skos  |    0.256     |           0.072            |
|     csc     |        2         |   bzip  |    0.094     |           0.003            |
|     csr     |        2         |   bzip  |    0.096     |           0.012            |
|     csc     |        2         | funding |    0.337     |           0.008            |
|     csr     |        2         | funding |    0.329     |           0.006            |
|     csc     |        2         |   gzip  |    0.899     |           0.011            |
|     csr     |        2         |   gzip  |    0.897     |           0.011            |

Из полученных данных можно видеть, что на выбранных графах в алгоритмах, реализованных мною для выполнения первых 4 домашних работ, нет существенной разницы, какой вид матриц применять для умножения. Для большинства графов различие в среднем времени выполнения с различными видами матриц связано со случайным выбросом, о чем можно судить по относительно большой величине среднего отклонение.

### Начиная с какого размера начального множества задачу нахождения всех пар достижимых вершин выгоднее решать используя 1 алгоритм, чем 2 алгоритм?

Для ответа на поставленный вопрос проведем замеры, где будем брать различное число начальных вершин в каждом графе (будем брать от 5 % вершин, до 55 % вершин графа для теста):

In [12]:
size_of_start_nodes_in_percent = [i / 100 for i in range(5, 60, 25)]

results_regular_path_query = []
results_bfs_based_regular_path_query = []
for percent in size_of_start_nodes_in_percent:
    for i in range(len(test_graphs)):
        graph = test_graphs[i]
        for regex in test_regexes[i]:
            start_states = list(cfpq_data.generate_multiple_source(graph, int(len(graph.nodes) * percent)))
            results_regular_path_query.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_regular_path_query(regex, graph, start_states=start_states)), percent))
            results_bfs_based_regular_path_query.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_bfs_based_regular_path_query(regex, graph, separated=True, start_states=start_states)), percent))

[2022-10-14 15:18:02]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 15:18:46]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 15:18:56]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 15:19:09]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8560763310> for multiple-source evaluation
[2022-10-14 15:19:58]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8560763310> for multiple-source evaluation
[2022-10-14 15:20:31]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.cl

И распечатаем результат:

In [13]:
-rows = []
for i in range(len(results_regular_path_query)):
    data = results_regular_path_query[i]
    rows.append(["1", data[0], data[3], ".*" if len(data[1]) > 30 else data[1].replace("|", "/"), '%.3f' %data[2][0], '%.3f' %data[2][1]])
    data = results_bfs_based_regular_path_query[i]
    rows.append(["2", data[0], data[3], ".*" if len(data[1]) > 30 else data[1].replace("|", "/"), '%.3f' %data[2][0], '%.3f' %data[2][1]])

print_table(["Algorithm number", "Graph", "Fraction of graph vertices per test", "Regex", "Mean of time", "Standard deviation of time"], rows)

| Algorithm number |  Graph  | Fraction of graph vertices per test |          Regex           | Mean of time | Standard deviation of time |
|:----------------:|:-------:|:-----------------------------------:|:------------------------:|:------------:|:--------------------------:|
|        1         |   skos  |                 0.05                |            .*            |    0.139     |           0.005            |
|        2         |   skos  |                 0.05                |            .*            |    0.301     |           0.002            |
|        1         |   skos  |                 0.05                |         unionOf          |    0.024     |           0.002            |
|        2         |   skos  |                 0.05                |         unionOf          |    0.074     |           0.001            |
|        1         |   skos  |                 0.05                |            .*            |    0.029     |           0.002            |
|        2         |

##### Результат полученный автором
| Algorithm number |  Graph  | Fraction of graph vertices per test |          Regex           | Mean of time | Standard deviation of time |
|:----------------:|:-------:|:-----------------------------------:|:------------------------:|:------------:|:--------------------------:|
|        1         |   skos  |                 0.05                |            .*            |    0.139     |           0.005            |
|        2         |   skos  |                 0.05                |            .*            |    0.301     |           0.002            |
|        1         |   skos  |                 0.05                |         unionOf          |    0.024     |           0.002            |
|        2         |   skos  |                 0.05                |         unionOf          |    0.074     |           0.001            |
|        1         |   skos  |                 0.05                |            .*            |    0.029     |           0.002            |
|        2         |   skos  |                 0.05                |            .*            |    0.102     |           0.002            |
|        1         |   bzip  |                 0.05                |         ((a/d))*         |    0.208     |           0.005            |
|        2         |   bzip  |                 0.05                |         ((a/d))*         |    0.283     |           0.002            |
|        1         |   bzip  |                 0.05                |            a             |    0.143     |           0.005            |
|        2         |   bzip  |                 0.05                |            a             |    0.190     |           0.002            |
|        1         |   bzip  |                 0.05                |       (d.((d)*.a))       |    0.544     |           0.010            |
|        2         |   bzip  |                 0.05                |       (d.((d)*.a))       |    0.293     |           0.032            |
|        1         | funding |                 0.05                |            .*            |    0.535     |           0.011            |
|        2         | funding |                 0.05                |            .*            |    1.618     |           0.098            |
|        1         | funding |                 0.05                |       description        |    0.142     |           0.004            |
|        2         | funding |                 0.05                |       description        |    0.320     |           0.009            |
|        1         | funding |                 0.05                | (rest.((rest)*.creator)) |    0.135     |           0.003            |
|        2         | funding |                 0.05                | (rest.((rest)*.creator)) |    0.342     |           0.034            |
|        1         |   gzip  |                 0.05                |         ((a/d))*         |    3.360     |           0.054            |
|        2         |   gzip  |                 0.05                |         ((a/d))*         |    1.813     |           0.023            |
|        1         |   gzip  |                 0.05                |            d             |    2.915     |           0.048            |
|        2         |   gzip  |                 0.05                |            d             |    1.459     |           0.015            |
|        1         |   gzip  |                 0.05                |       (d.((d)*.a))       |    11.947    |           0.700            |
|        2         |   gzip  |                 0.05                |       (d.((d)*.a))       |    2.038     |           0.040            |
|        1         |   skos  |                 0.3                 |            .*            |    0.167     |           0.015            |
|        2         |   skos  |                 0.3                 |            .*            |    1.544     |           0.043            |
|        1         |   skos  |                 0.3                 |         unionOf          |    0.028     |           0.004            |
|        2         |   skos  |                 0.3                 |         unionOf          |    0.209     |           0.011            |
|        1         |   skos  |                 0.3                 |            .*            |    0.035     |           0.005            |
|        2         |   skos  |                 0.3                 |            .*            |    0.491     |           0.019            |
|        1         |   bzip  |                 0.3                 |         ((a/d))*         |    0.255     |           0.013            |
|        2         |   bzip  |                 0.3                 |         ((a/d))*         |    1.757     |           0.020            |
|        1         |   bzip  |                 0.3                 |            a             |    0.175     |           0.015            |
|        2         |   bzip  |                 0.3                 |            a             |    0.875     |           0.018            |
|        1         |   bzip  |                 0.3                 |       (d.((d)*.a))       |    0.648     |           0.014            |
|        2         |   bzip  |                 0.3                 |       (d.((d)*.a))       |    1.750     |           0.020            |
|        1         | funding |                 0.3                 |            .*            |    0.643     |           0.013            |
|        2         | funding |                 0.3                 |            .*            |    6.612     |           0.037            |
|        1         | funding |                 0.3                 |       description        |    0.156     |           0.005            |
|        2         | funding |                 0.3                 |       description        |    1.203     |           0.017            |
|        1         | funding |                 0.3                 | (rest.((rest)*.creator)) |    0.149     |           0.014            |
|        2         | funding |                 0.3                 | (rest.((rest)*.creator)) |    2.013     |           0.024            |
|        1         |   gzip  |                 0.3                 |         ((a/d))*         |    3.965     |           0.057            |
|        2         |   gzip  |                 0.3                 |         ((a/d))*         |    8.229     |           0.077            |
|        1         |   gzip  |                 0.3                 |            d             |    3.297     |           0.040            |
|        2         |   gzip  |                 0.3                 |            d             |    5.807     |           0.042            |
|        1         |   gzip  |                 0.3                 |       (d.((d)*.a))       |    12.984    |           0.246            |
|        2         |   gzip  |                 0.3                 |       (d.((d)*.a))       |    10.205    |           0.080            |
|        1         |   skos  |                 0.55                |            .*            |    0.158     |           0.005            |
|        2         |   skos  |                 0.55                |            .*            |    2.820     |           0.026            |
|        1         |   skos  |                 0.55                |         unionOf          |    0.025     |           0.003            |
|        2         |   skos  |                 0.55                |         unionOf          |    0.375     |           0.010            |
|        1         |   skos  |                 0.55                |            .*            |    0.033     |           0.003            |
|        2         |   skos  |                 0.55                |            .*            |    0.760     |           0.224            |
|        1         |   bzip  |                 0.55                |         ((a/d))*         |    0.304     |           0.009            |
|        2         |   bzip  |                 0.55                |         ((a/d))*         |    3.541     |           0.078            |
|        1         |   bzip  |                 0.55                |            a             |    0.204     |           0.008            |
|        2         |   bzip  |                 0.55                |            a             |    1.849     |           0.047            |
|        1         |   bzip  |                 0.55                |       (d.((d)*.a))       |    0.762     |           0.024            |
|        2         |   bzip  |                 0.55                |       (d.((d)*.a))       |    3.723     |           0.059            |
|        1         | funding |                 0.55                |            .*            |    0.748     |           0.057            |
|        2         | funding |                 0.55                |            .*            |    15.192    |           0.169            |
|        1         | funding |                 0.55                |       description        |    0.174     |           0.008            |
|        2         | funding |                 0.55                |       description        |    2.502     |           0.050            |
|        1         | funding |                 0.55                | (rest.((rest)*.creator)) |    0.161     |           0.006            |
|        2         | funding |                 0.55                | (rest.((rest)*.creator)) |    4.258     |           0.133            |
|        1         |   gzip  |                 0.55                |         ((a/d))*         |    4.933     |           0.080            |
|        2         |   gzip  |                 0.55                |         ((a/d))*         |    17.362    |           0.167            |
|        1         |   gzip  |                 0.55                |            d             |    4.129     |           0.070            |
|        2         |   gzip  |                 0.55                |            d             |    14.106    |           0.164            |
|        1         |   gzip  |                 0.55                |       (d.((d)*.a))       |    16.377    |           0.364            |
|        2         |   gzip  |                 0.55                |       (d.((d)*.a))       |    26.662    |           0.235            |

Из полученных данных можно видеть, что на выбранных графах реализованный мной 2 алгоритм в среднем показывает себя хуже на любом количестве начальных вершин, даже маленьком. Только на графе gzip при маленьком количестве начальных вершин и на определенных регулярных выражениях получилось выиграть по скорости алгоритмом 2 у алгоритма 1, однако наиболее вероятно это просто связано с удачно выбранным регулярным выражением, которое не привело к долгому поиску в ширину.

### На сколько реализация 2 алгоритм медленнее работает, если необходимо найти множества достижимых вершин для каждой из вершин начального множества?

Для ответа на поставленный вопрос проведем замеры, где будем брать различное число вершин в каждом графе (будем брать от 5 % вершин, до 55 % вершин для теста):

In [14]:
size_of_start_nodes_in_percent = [i / 100 for i in range(5, 60, 25)]

results_separated_true = []
results_separated_false = []
for percent in size_of_start_nodes_in_percent:
    for i in range(len(test_graphs)):
        graph = test_graphs[i]
        for regex in test_regexes[i]:
            start_states = list(cfpq_data.generate_multiple_source(graph, int(len(graph.nodes) * percent)))
            results_separated_true.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_bfs_based_regular_path_query(regex, graph, separated=True, start_states=start_states)), percent))
            results_separated_false.append((test_graph_names[i], str(regex), time(lambda: experiment_pass_bfs_based_regular_path_query(regex, graph, separated=False, start_states=start_states)), percent))

[2022-10-14 21:13:34]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 21:14:31]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 21:14:44]>INFO>Generate set of source vertices of 7 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8569a1e8c0> for multiple-source evaluation
[2022-10-14 21:15:05]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8560763310> for multiple-source evaluation
[2022-10-14 21:15:40]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.classes.multidigraph.MultiDiGraph object at 0x7f8560763310> for multiple-source evaluation
[2022-10-14 21:16:08]>INFO>Generate set of source vertices of 31 nodes for graph=<networkx.cl

И распечатаем результат (после Separated отвечает выбору режима работы второго алгоритма. Если Separated == true, то необходимо найти множества достижимых вершин для каждой из вершин начального множества):

In [15]:
rows = []
for i in range(len(results_separated_true)):
    data = results_separated_true[i]
    rows.append(["true", data[0], data[3], ".*" if len(data[1]) > 30 else data[1].replace("|", "/"), '%.3f' %data[2][0], '%.3f' %data[2][1]])
    data = results_separated_false[i]
    rows.append(["false", data[0], data[3], ".*" if len(data[1]) > 30 else data[1].replace("|", "/"), '%.3f' %data[2][0], '%.3f' %data[2][1]])

print_table(["Separated", "Graph", "Fraction of graph vertices per test", "Regex", "Mean of time", "Standard deviation of time"], rows)

| Separated |  Graph  | Fraction of graph vertices per test |          Regex           | Mean of time | Standard deviation of time |
|:---------:|:-------:|:-----------------------------------:|:------------------------:|:------------:|:--------------------------:|
|    true   |   skos  |                 0.05                |            .*            |    0.349     |           0.008            |
|   false   |   skos  |                 0.05                |            .*            |    0.214     |           0.003            |
|    true   |   skos  |                 0.05                |         unionOf          |    0.073     |           0.003            |
|   false   |   skos  |                 0.05                |         unionOf          |    0.058     |           0.001            |
|    true   |   skos  |                 0.05                |            .*            |    0.123     |           0.003            |
|   false   |   skos  |                 0.05                |        

##### Результат полученный автором
| Separated |  Graph  | Fraction of graph vertices per test |          Regex           | Mean of time | Standard deviation of time |
|:---------:|:-------:|:-----------------------------------:|:------------------------:|:------------:|:--------------------------:|
|    true   |   skos  |                 0.05                |            .*            |    0.349     |           0.008            |
|   false   |   skos  |                 0.05                |            .*            |    0.214     |           0.003            |
|    true   |   skos  |                 0.05                |         unionOf          |    0.073     |           0.003            |
|   false   |   skos  |                 0.05                |         unionOf          |    0.058     |           0.001            |
|    true   |   skos  |                 0.05                |            .*            |    0.123     |           0.003            |
|   false   |   skos  |                 0.05                |            .*            |    0.087     |           0.001            |
|    true   |   bzip  |                 0.05                |         ((a/d))*         |    0.238     |           0.004            |
|   false   |   bzip  |                 0.05                |         ((a/d))*         |    0.109     |           0.003            |
|    true   |   bzip  |                 0.05                |            a             |    0.195     |           0.006            |
|   false   |   bzip  |                 0.05                |            a             |    0.092     |           0.001            |
|    true   |   bzip  |                 0.05                |       (d.((d)*.a))       |    0.326     |           0.004            |
|   false   |   bzip  |                 0.05                |       (d.((d)*.a))       |    0.110     |           0.001            |
|    true   | funding |                 0.05                |            .*            |    1.103     |           0.008            |
|   false   | funding |                 0.05                |            .*            |    0.336     |           0.002            |
|    true   | funding |                 0.05                |       description        |    0.320     |           0.005            |
|   false   | funding |                 0.05                |       description        |    0.197     |           0.002            |
|    true   | funding |                 0.05                | (rest.((rest)*.creator)) |    0.332     |           0.005            |
|   false   | funding |                 0.05                | (rest.((rest)*.creator)) |    0.187     |           0.002            |
|    true   |   gzip  |                 0.05                |         ((a/d))*         |    1.985     |           0.010            |
|   false   |   gzip  |                 0.05                |         ((a/d))*         |    0.963     |           0.014            |
|    true   |   gzip  |                 0.05                |            d             |    1.373     |           0.008            |
|   false   |   gzip  |                 0.05                |            d             |    0.901     |           0.006            |
|    true   |   gzip  |                 0.05                |       (d.((d)*.a))       |    1.934     |           0.009            |
|   false   |   gzip  |                 0.05                |       (d.((d)*.a))       |    0.922     |           0.007            |
|    true   |   skos  |                 0.3                 |            .*            |    1.335     |           0.009            |
|   false   |   skos  |                 0.3                 |            .*            |    0.224     |           0.002            |
|    true   |   skos  |                 0.3                 |         unionOf          |    0.178     |           0.004            |
|   false   |   skos  |                 0.3                 |         unionOf          |    0.063     |           0.001            |
|    true   |   skos  |                 0.3                 |            .*            |    0.418     |           0.005            |
|   false   |   skos  |                 0.3                 |            .*            |    0.108     |           0.001            |
|    true   |   bzip  |                 0.3                 |         ((a/d))*         |    1.624     |           0.024            |
|   false   |   bzip  |                 0.3                 |         ((a/d))*         |    0.166     |           0.002            |
|    true   |   bzip  |                 0.3                 |            a             |    0.818     |           0.007            |
|   false   |   bzip  |                 0.3                 |            a             |    0.129     |           0.002            |
|    true   |   bzip  |                 0.3                 |       (d.((d)*.a))       |    1.633     |           0.016            |
|   false   |   bzip  |                 0.3                 |       (d.((d)*.a))       |    0.149     |           0.001            |
|    true   | funding |                 0.3                 |            .*            |    8.224     |           0.364            |
|   false   | funding |                 0.3                 |            .*            |    0.429     |           0.009            |
|    true   | funding |                 0.3                 |       description        |    1.197     |           0.018            |
|   false   | funding |                 0.3                 |       description        |    0.251     |           0.006            |
|    true   | funding |                 0.3                 | (rest.((rest)*.creator)) |    2.000     |           0.021            |
|   false   | funding |                 0.3                 | (rest.((rest)*.creator)) |    0.253     |           0.005            |
|    true   |   gzip  |                 0.3                 |         ((a/d))*         |    8.264     |           0.055            |
|   false   |   gzip  |                 0.3                 |         ((a/d))*         |    1.369     |           0.018            |
|    true   |   gzip  |                 0.3                 |            d             |    5.807     |           0.051            |
|   false   |   gzip  |                 0.3                 |            d             |    1.229     |           0.013            |
|    true   |   gzip  |                 0.3                 |       (d.((d)*.a))       |    11.236    |           0.102            |
|   false   |   gzip  |                 0.3                 |       (d.((d)*.a))       |    1.359     |           0.019            |
|    true   |   skos  |                 0.55                |            .*            |    2.845     |           0.029            |
|   false   |   skos  |                 0.55                |            .*            |    0.250     |           0.006            |
|    true   |   skos  |                 0.55                |         unionOf          |    0.312     |           0.007            |
|   false   |   skos  |                 0.55                |         unionOf          |    0.073     |           0.002            |
|    true   |   skos  |                 0.55                |            .*            |    0.646     |           0.010            |
|   false   |   skos  |                 0.55                |            .*            |    0.094     |           0.004            |
|    true   |   bzip  |                 0.55                |         ((a/d))*         |    2.954     |           0.037            |
|   false   |   bzip  |                 0.55                |         ((a/d))*         |    0.187     |           0.005            |
|    true   |   bzip  |                 0.55                |            a             |    1.686     |           0.022            |
|   false   |   bzip  |                 0.55                |            a             |    0.169     |           0.004            |
|    true   |   bzip  |                 0.55                |       (d.((d)*.a))       |    3.367     |           0.029            |
|   false   |   bzip  |                 0.55                |       (d.((d)*.a))       |    0.205     |           0.006            |
|    true   | funding |                 0.55                |            .*            |    13.746    |           0.076            |
|   false   | funding |                 0.55                |            .*            |    0.445     |           0.009            |
|    true   | funding |                 0.55                |       description        |    2.270     |           0.027            |
|   false   | funding |                 0.55                |       description        |    0.293     |           0.008            |
|    true   | funding |                 0.55                | (rest.((rest)*.creator)) |    3.305     |           0.042            |
|   false   | funding |                 0.55                | (rest.((rest)*.creator)) |    0.294     |           0.008            |
|    true   |   gzip  |                 0.55                |         ((a/d))*         |    15.713    |           0.096            |
|   false   |   gzip  |                 0.55                |         ((a/d))*         |    1.600     |           0.024            |
|    true   |   gzip  |                 0.55                |            d             |    12.769    |           0.061            |
|   false   |   gzip  |                 0.55                |            d             |    1.510     |           0.016            |
|    true   |   gzip  |                 0.55                |       (d.((d)*.a))       |    24.228    |           0.131            |
|   false   |   gzip  |                 0.55                |       (d.((d)*.a))       |    1.671     |           0.028            |

Из полученных данных можно видеть, что на выбранных графах реализованный мной 2 алгоритм в режиме "нахождения достижимых вершин из множества начальных" (далее совместный режим) работает быстрее, чем в режиме "нахождения множества достижимых вершин для каждой вершины из множества начальных" (далее раздельный режим). Это ожидаемый результат, так как в "совместном" режиме на каждой итерации алгоритма выполняется намного меньше матричных умножений, а также другие матричные операции работают быстрее в силу меньшего размера самих используемых матриц.